feature extractor:

    input: route (
    output: features
            

In [1]:
import pandas as pd
import numpy as np
import ast

In [2]:
# initiate problems dataframe
# read from csv file
# cast it into Pandas DataFrame
problems = pd.read_csv("problems.csv")
problems.drop(["Unnamed: 0"], axis=1, inplace=True)
problems.head()

,Name,Grade,Moves,Holdsets,Id
0,BLACK BEAUTY,8B,"[{'Id': 1542554, 'Description': 'J5', 'IsStart...","[{'Id': 0, 'Description': 'Original School Hol...",82224
1,PROJECT 2,8A+,"[{'Id': 1535606, 'Description': 'K6', 'IsStart...","[{'Id': 0, 'Description': 'Original School Hol...",62575
2,SHATTERED MERCY,8A+,"[{'Id': 1596473, 'Description': 'D3', 'IsStart...","[{'Id': 0, 'Description': 'Original School Hol...",241045
3,ATTIC ADDICT,8A,"[{'Id': 1986894, 'Description': 'E6', 'IsStart...","[{'Id': 0, 'Description': 'Original School Hol...",350783
4,BUNNY,8A,"[{'Id': 1521147, 'Description': 'B4', 'IsStart...","[{'Id': 0, 'Description': 'Original School Hol...",23184


In [3]:
# Converts type of elements of Moves columns into python list

problem_list = []
for problem in problems["Moves"].values:
    problem_as_lst = ast.literal_eval(problem)
    problem_list.append(problem_as_lst)
problems["Moves"] = problem_list   

In [4]:
# feature 1: number of holds
# extracts number of holds in route and write it into column "Number of Holds"

number_of_holds = []
for problem in problems["Moves"].values:
    number_of_holds.append(len(problem))
problems["numberOfHolds"] = number_of_holds
problems.head()


,Name,Grade,Moves,Holdsets,Id,numberOfHolds
0,BLACK BEAUTY,8B,"[{'Id': 1542554, 'Description': 'J5', 'IsStart...","[{'Id': 0, 'Description': 'Original School Hol...",82224,4
1,PROJECT 2,8A+,"[{'Id': 1535606, 'Description': 'K6', 'IsStart...","[{'Id': 0, 'Description': 'Original School Hol...",62575,6
2,SHATTERED MERCY,8A+,"[{'Id': 1596473, 'Description': 'D3', 'IsStart...","[{'Id': 0, 'Description': 'Original School Hol...",241045,8
3,ATTIC ADDICT,8A,"[{'Id': 1986894, 'Description': 'E6', 'IsStart...","[{'Id': 0, 'Description': 'Original School Hol...",350783,8
4,BUNNY,8A,"[{'Id': 1521147, 'Description': 'B4', 'IsStart...","[{'Id': 0, 'Description': 'Original School Hol...",23184,8


In [5]:
# feature 2: longest eucledian distance between two holds of a route
# extracts the longest move in a route



# returns number of start holds in a route
def areTwoStartHolds(problem):
    number_of_start_holds = 0
    for move in problem:
        if move["IsStart"]:
            number_of_start_holds += 1
            if number_of_start_holds == 2:
                break
    return number_of_start_holds

# list of alphabet
alphabet = ["A","B","C","D","E","F","G","H","I","J","K","L","M","N","O","P","Q","R","S","T","U","V","W","X","Y","Z"]


# returns euclidean distance between two holds:
def eucl_dist(hold1, hold2):
    # hold1 as numpy array
    x_coord = alphabet.index(hold1[0].upper())
    y_coord = int(hold1[1:])
    hold1_tuple = tuple((x_coord, y_coord))
    hold1 = np.array(hold1_tuple)
    
    # hold2 as numpy array
    x_coord = alphabet.index(hold2[0].upper())
    y_coord = int(hold2[1:])
    hold2_tuple = tuple((x_coord, y_coord))  
    hold2 = np.array(hold2_tuple)   
    
    dist = np.linalg.norm(hold1-hold2)
    
    return dist

# returns list of move lengths in a route
def len_of_moves(problem):
    len_of_moves = []
    
    num_of_moves = len(problem["Moves"])
    for i in range(num_of_moves - 1):
        hold1 = problem["Moves"][i]["Description"]
        hold2 = problem["Moves"][i+1]["Description"]
        len_of_moves.append(eucl_dist(hold1,hold2))
    
    return len_of_moves    


# adds len_of_moves feature column

def len_feature(DataFrame):
    lst_of_len_of_moves = []

    for i in range(len(DataFrame)):
        lst_of_len_of_moves.append(len_of_moves(DataFrame.iloc[i]))
    DataFrame["lengthOfMoves"] = lst_of_len_of_moves
    return DataFrame


In [6]:
problems = len_feature(problems)

In [12]:
problems.head()

,Name,Grade,Moves,Holdsets,Id,numberOfHolds,lengthOfMoves
0,BLACK BEAUTY,8B,"[{'Id': 1542554, 'Description': 'J5', 'IsStart...","[{'Id': 0, 'Description': 'Original School Hol...",82224,4,"[4.123105625617661, 6.4031242374328485, 5.0]"
1,PROJECT 2,8A+,"[{'Id': 1535606, 'Description': 'K6', 'IsStart...","[{'Id': 0, 'Description': 'Original School Hol...",62575,6,"[1.0, 7.615773105863909, 5.656854249492381, 7...."
2,SHATTERED MERCY,8A+,"[{'Id': 1596473, 'Description': 'D3', 'IsStart...","[{'Id': 0, 'Description': 'Original School Hol...",241045,8,"[2.23606797749979, 5.830951894845301, 2.236067..."
3,ATTIC ADDICT,8A,"[{'Id': 1986894, 'Description': 'E6', 'IsStart...","[{'Id': 0, 'Description': 'Original School Hol...",350783,8,"[4.47213595499958, 5.0, 4.47213595499958, 5.65..."
4,BUNNY,8A,"[{'Id': 1521147, 'Description': 'B4', 'IsStart...","[{'Id': 0, 'Description': 'Original School Hol...",23184,8,"[5.830951894845301, 1.0, 3.1622776601683795, 2..."
